In [1]:
!pip install --quiet huggingface-hub llama-cpp-python langchain-community langchain chromadb sentence-transformers gspread google-auth google-auth-oauthlib google-auth-httplib2 gspread_dataframe pandas==2.2.2 openpyxl google-api-python-client

In [2]:
from google.colab import auth
auth.authenticate_user()

import json, re, os
from datetime import datetime, timedelta
import pytz
import pandas as pd

# Google API クライアント
# from googleapiclient.discovery import build # Calendar service import removed
import gspread
from gspread_dataframe import set_with_dataframe, get_as_dataframe
from google.auth import default

creds, _ = default()
# Calendar service # Calendar service initialization removed
# Sheets (gspread)
gc = gspread.authorize(creds)

print("✅ Google 認証完了")
# print("Calendar service and gspread client ready.") # Calendar service ready print removed
print("gspread client ready for Google Sheets.")

✅ Google 認証完了
gspread client ready for Google Sheets.


In [3]:
from huggingface_hub import hf_hub_download
from langchain_community.llms import LlamaCpp
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# --- 設定（必要に応じて変更） ---
CONTEXT_SIZE = 2048
LLM_REPO_ID = "mmnga/ELYZA-japanese-Llama-2-7b-instruct-gguf"
LLM_FILE = "ELYZA-japanese-Llama-2-7b-instruct-q4_K_S.gguf"

# hf_hub からモデルファイルをダウンロード（既にある場合はスキップしてパスを直接指定してもOK）
try:
    model_path = hf_hub_download(repo_id=LLM_REPO_ID, filename=LLM_FILE)
except Exception as e:
    print("モデルのダウンロードでエラー（もしくはローカルに既に置いてください）:", e)
    model_path = None

# Llama の初期化（model_path が None の場合は別途モデルパスを与えてください）
if model_path:
    llm = LlamaCpp(
        model_path=model_path,
        n_gpu_layers=0,   # Colab 環境に合わせて調整
        n_ctx=CONTEXT_SIZE,
        f16_kv=True,
        verbose=False,
        seed=0
    )
else:
    llm = None
    print("LLM が利用できません。対話モードは限定動作になります。")

# シンプルなプロンプトテンプレート
template = """あなたは日本語で丁寧に応答するアシスタントです。
ユーザーの依頼に基づき、必要であれば以下の `ACTION: {{ ... }}` 形式で操作指示を出力してください。
**重要**: プログラムに操作を実行させるには、必ずこのACTION形式を含めてください。

有効な action 値と必要な params:
- read_sheet (params: spreadsheet_name, sheet_name (任意))
- add_sheet_row (params: spreadsheet_name, sheet_name (任意), row (リスト))
- update_sheet_row (params: spreadsheet_name, sheet_name (任意), row_index (1-based), row (リスト))
- delete_sheet_row (params: spreadsheet_name, sheet_name (任意), row_index (1-based))
- **update_sheet_cell_by_date** (params: spreadsheet_name, sheet_name (任意), date (YYYY-MM-DD形式), column_index (現在3のみ対応), value (更新する値))
  - **注**: このアクションは、スプレッドシートが2行目から2025年1月1日を開始とし、日付が下に続く形式であることを想定しています。指定された日付に基づいて自動的に行番号を計算し、指定された列（現在は3列目のみ）を更新します。スプレッドシート名は固定（"schedule_calendar_2025"）で処理されますが、ACTIONには含めても構いません。

**ACTION 生成の例**:
ユーザー: schedule_calendar_2025 の 2025-01-05 の行の 3 列目を 休暇 に更新して
アシスタント: 承知いたしました。以下のACTIONを実行します。
ACTION: {{ "action": "update_sheet_cell_by_date", "params": {{ "spreadsheet_name":"schedule_calendar_2025", "date":"2025-01-05", "column_index":3, "value":"休暇" }} }}

ユーザー: {input}
アシスタント:
"""

prompt = ChatPromptTemplate.from_template(template)
output_parser = StrOutputParser()

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
llama_context: n_batch is less than GGML_KQ_MASK_PAD - increasing to 64
llama_context: n_ctx_per_seq (2048) < n_ctx_train (4096) -- the full capacity of the model will not be utilized


In [4]:
# --- Calendar helpers --- # Calendar helpers removed
# def list_events(...): ...
# def add_event(...): ...
# def update_event(...): ...
# def delete_event(...): ...

# --- Sheets helpers (gspread) ---
def open_or_create_spreadsheet(name):
    try:
        sh = gc.open(name)
    except Exception:
        sh = gc.create(name)
        # 共有設定を変えたい場合はここで： sh.share('...', perm_type='user', role='writer')
    return sh

def read_sheet(spreadsheet_name, sheet_name=None):
    sh = open_or_create_spreadsheet(spreadsheet_name)
    if sheet_name:
        try:
            ws = sh.worksheet(sheet_name)
        except Exception:
            ws = sh.add_worksheet(title=sheet_name, rows=1000, cols=20)
    else:
        ws = sh.sheet1
    df = get_as_dataframe(ws, evaluate_formulas=True, header=0)
    return df.fillna("")

def add_sheet_row(spreadsheet_name, sheet_name, row_values):
    sh = open_or_create_spreadsheet(spreadsheet_name)
    try:
        ws = sh.worksheet(sheet_name)
    except Exception:
        ws = sh.add_worksheet(title=sheet_name, rows=1000, cols=20)
    ws.append_row(row_values)
    return True

def update_sheet_row(spreadsheet_name, sheet_name, row_index, row_values):
    sh = open_or_create_spreadsheet(spreadsheet_name)
    ws = sh.worksheet(sheet_name)
    # gspread uses 1-based indexing for rows
    for col, val in enumerate(row_values, start=1):
        ws.update_cell(row_index, col, val)
    return True

def delete_sheet_row(spreadsheet_name, sheet_name, row_index):
    sh = open_or_create_spreadsheet(spreadsheet_name)
    ws = sh.worksheet(sheet_name)
    ws.delete_rows(row_index)
    return True

In [11]:
import ast
from datetime import datetime, timedelta # datetimeとtimedeltaをインポート
import re # 正規表現モジュールをインポート

def extract_action(text):
    """
    テキスト中の ACTION: {...} を探して JSON 部分を返す（見つからなければ None）。
    LLM が辞書形式を返すケースにもあるため、Python 文法風にも対応。
    """
    m = re.search(r"ACTION\s*:\s*({.*?})\s*$", text, flags=re.S)
    # 末尾でない場合も探す（最初のマッチ）
    if not m:
        m = re.search(r"ACTION\s*:\s*({.*?})", text, flags=re.S)
    if not m:
        return None
    json_text = m.group(1)
    # JSONとしてパースを試みる（シングルクォート等も許容する）
    try:
        return json.loads(json_text)
    except Exception:
        try:
            # Python 表記の dict を ast.literal_eval でパース
            return ast.literal_eval(json_text)
        except Exception:
            # 微修正してみる（改行や末尾のカンマの除去など）
            cleaned = re.sub(r",\s*}", "}", json_text)
            cleaned = re.sub(r",\s*]", "]", cleaned)
            try:
                return json.loads(cleaned)
            except Exception:
                try:
                    return ast.literal_eval(cleaned)
                except Exception:
                    return None


def update_sheet_cell(spreadsheet_name, sheet_name, row_index, col_index, value):
    """指定したスプレッドシートの特定のセルを更新するヘルパー関数"""
    sh = open_or_create_spreadsheet(spreadsheet_name)
    # sheet_nameがNoneの場合は最初のシートを取得
    ws = sh.sheet1 if sheet_name is None else sh.worksheet(sheet_name)
    ws.update_cell(row_index, col_index, value)
    return True


def perform_action(action_obj):
    """
    action_obj: {'action': 'add_event', 'params': {...}}
    """
    if not isinstance(action_obj, dict) or 'action' not in action_obj:
        return {"status":"error","message":"無効なACTION形式"}
    act = action_obj['action']
    params = action_obj.get('params', {})

    # スプレッドシート名を固定
    FIXED_SPREADSHEET_NAME = "schedule_calendar_2025"

    try:
        # Calendar actions removed
        # if act == 'add_event':
        #     created = add_event(
        #         summary=params['summary'],
        #         start=params['start'],
        #         end=params['end'],
        #         description=params.get('description', "")
        #     )
        #     return {"status":"ok","result":created}
        # elif act == 'list_events':
        #     items = list_events(time_min=params.get('timeMin'), time_max=params.get('timeMax'), maxResults=params.get('maxResults',10))
        #     return {"status":"ok","result":items}
        # elif act == 'update_event':
        #     updated = update_event(params['eventId'], params.get('updates', {}))
        #     return {"status":"ok","result":updated}
        # elif act == 'delete_event':
        #     delete_event(params['eventId'])
        #     return {"status":"ok","result":"deleted"}
        if act == 'read_sheet':
            df = read_sheet(FIXED_SPREADSHEET_NAME, params.get('sheet_name')) # 固定名を使用
            return {"status":"ok","result": df.to_dict(orient='records')}
        elif act == 'add_sheet_row':
            add_sheet_row(FIXED_SPREADSHEET_NAME, params['sheet_name'], params['row']) # 固定名を使用
            return {"status":"ok","result":"row_added"}
        elif act == 'update_sheet_row':
            update_sheet_row(FIXED_SPREADSHEET_NAME, params['sheet_name'], params['row_index'], params['row']) # 固定名を使用
            return {"status":"ok","result":"row_updated"}
        elif act == 'delete_sheet_row':
            sheet_name = params.get('sheet_name') # 安全に取得
            row_index = params.get('row_index') # 安全に取得
            if row_index is None:
                 return {"status":"error","message":"'delete_sheet_row'には 'row_index' が必要です。"}
            # row_indexが数値かどうかのバリデーションは行わない（LLMの生成に依存）
            delete_sheet_row(FIXED_SPREADSHEET_NAME, sheet_name, row_index) # 安全なシート名と行インデックスを渡す
            return {"status":"ok","result":"row_deleted"}
        # 新しいアクションハンドリング
        elif act == 'update_sheet_cell_by_date':
            # spreadsheet_name = params.get('spreadsheet_name') # パラメータから取得せず固定
            sheet_name = params.get('sheet_name')
            date_str = params.get('date')
            column_index = params.get('column_index')
            value = params.get('value')

            # 固定されたスプレッドシート名を使用
            spreadsheet_name_to_use = FIXED_SPREADSHEET_NAME


            if not all([date_str, column_index is not None, value is not None]):
                 return {"status":"error","message":"'update_sheet_cell_by_date'には 'date', 'column_index', 'value' が必要です（スプレッドシート名は固定されています）。"}

            if column_index != 3:
                 return {"status":"error","message":"'update_sheet_cell_by_date'は現在、3列目のみに対応しています。"}

            # 日付から行番号を計算 (2025-01-01 が 2行目)
            start_date = datetime(2025, 1, 1)
            start_row_index = 2
            try:
                target_date = datetime.strptime(date_str, '%Y-%m-%d')
                days_difference = (target_date - start_date).days
                if days_difference < 0:
                     return {"status":"error","message":"指定された日付が開始日(2025-01-01)より前です。"}
                target_row_index = start_row_index + days_difference
            except ValueError:
                 return {"status":"error","message":"'date'の形式が無効です。'YYYY-MM-DD'形式で指定してください。"}

            print(f"スプレッドシート '{spreadsheet_name_to_use}', シート '{sheet_name or '最初のシート'}' の日付 '{date_str}' (行 {target_row_index}) の {column_index} 列目を '{value}' に更新中...")

            # 実際のセル更新処理 (gspreadは1-based index) - 固定名を使用
            # update_sheet_cell内でsheet_nameがNoneの場合の処理を記述したので、ここではそのまま渡す
            update_sheet_cell(spreadsheet_name_to_use, sheet_name, target_row_index, column_index, value)

            return {"status":"ok","result":f"日付 '{date_str}' の {column_index} 列目を更新しました (行 {target_row_index})。"}

        else:
            # If no valid action is found, indicate that.
            return {"status":"error","message":"未対応または不明なactionです: "+str(act)}
    except Exception as e:
        print(f"エラーが発生しました: {e}")
        return {"status":"error","message":str(e)}

# チャット関数
def chat_once(user_input):
    """
    ユーザー入力に応じて、直接スプレッドシート操作を行うか、LLMに投げて応答を得る。
    もし LLM が ACTION を返したら自動的に実行して結果を表示する。
    """
    global llm
    # 特定のユーザー入力パターンをチェックし、直接処理する
    FIXED_SPREADSHEET_NAME = "schedule_calendar_2025" # ここでも固定名を使用
    match = re.search(r"schedule_calendar_2025 の\s*\[?(\d{4}-\d{2}-\d{2})\]?\s*の行の\s*3\s*列目を\s*\[?(.*?)\]?\s*に更新して", user_input)

    if match:
        date_str = match.group(1)
        value = match.group(2).strip() # 前後の空白を除去

        print(f"== Direct Command Detected ==")
        print(f"日付: {date_str}, 値: {value} でスプレッドシート更新を試みます。")

        # update_sheet_cell_by_date アクションを直接構築して実行
        action_to_perform = {
            "action": "update_sheet_cell_by_date",
            "params": {
                "spreadsheet_name": FIXED_SPREADSHEET_NAME, # 固定名
                "date": date_str,
                "column_index": 3,
                "value": value
            }
        }
        res = perform_action(action_to_perform)
        print("\n-- Direct Action Execution Result --")
        print(json.dumps(res, ensure_ascii=False, indent=2))
        return # 直接処理したのでここで終了

    # 特定のパターンに一致しない場合は、通常のLLM処理に進む
    print("== Passing to Assistant ==")
    # プロンプトを準備
    prompt_text = template.format(input=user_input)

    # LLM がある場合は呼ぶ（簡易）
    if llm:
        try:
            # LlamaCpp wrapper in langchain_community may differ; here we call llm directly if possible
            # LLMに投げる前に、LLMが見るuser_inputから固定スプレッドシート名を削除するか検討
            # 今回はそのまま投げるが、LLMの応答によっては調整が必要になるかも
            resp = llm.invoke(prompt_text) # Recommended call
            text = resp
        except Exception as e:
            print(f"LLM実行エラー: {e}")
            # 代替：直接 print a fallback
            text = "申し訳ありません、LLM の実行に失敗しました。"
    else:
        # LLM が無い場合は簡単ルールで応答
        text = "（LLM が利用できないため、簡易応答）何をしたいですか？（例: 予定を追加、予定を一覧）"

    print("== Assistant Response ==")
    print(text)

    # ACTION 抽出 (LLMが生成した場合のみ)
    act = extract_action(text)
    if act:
        print("\n-- ACTION 検出: 実行します --")
        res = perform_action(act)
        print(json.dumps(res, ensure_ascii=False, indent=2))
    else:
        print("\n-- ACTION は見つかりませんでした（通常応答） --")

user_input = "schedule_calendar_2025 の 2025-01-03 の行の 3 列目を 講義 に更新して"
chat_once(user_input)

== Direct Command Detected ==
日付: 2025-01-03, 値: 講義 でスプレッドシート更新を試みます。
スプレッドシート 'schedule_calendar_2025', シート '最初のシート' の日付 '2025-01-03' (行 4) の 3 列目を '講義' に更新中...

-- Direct Action Execution Result --
{
  "status": "ok",
  "result": "日付 '2025-01-03' の 3 列目を更新しました (行 4)。"
}
